In [1]:
def generate_latex_table(data):
    latex_header = r"""
\begin{table}[]
    \centering
    \footnotesize
    \caption{Jetson Orin AGX}
    \begin{tabular}{@{}clccccccccccc@{}}
    \toprule
    & & \multicolumn{2}{c}{Latency {[}ms{]}} &  & Thr. {[}inf/s{]} &  \multicolumn{3}{c}{Model properties} &  & \multicolumn{2}{c}{Accuracy [\%]} \\ \cmidrule(lr){3-4}  \cmidrule(lr){7-9} \cmidrule(l){11-12} 
    & & ave.              & max.             &  &                       &  Size {[}MB{]}   & \# Layers   & \# Weights   &  & Top 1       & Top 5      \\ \midrule"""
    latex_footer = r"""
    \bottomrule
    \end{tabular}
\end{table}
    """

    latex_content = ""
    is_first = True
    for model_name, variants in data.items():
        if variants:
            model_section = f" "
            for i, variant in enumerate(variants):
                variant_name = variant['variant'].replace('_', ' ')  # Replace underscores with spaces
                if i == 0:
                    if is_first:
                        model_section += f" \\multirow{{4}}{{*}}{{\\rotatebox[origin=c]{{90}}{{{model_name}}}}}  "
                        model_section += f"& {variant_name} & {variant['lat_mean']} & {variant['lat_worst']} & & {variant['inf_s']} & {variant['size_mb']} & {variant['layers']} & {variant['params']} & & {variant['prec_1']} & {variant['prec_5']} \\\\"
                        is_first = False
                    else:
                        model_section += f"   \\midrule \n"
                        model_section += f"    \\multirow{{4}}{{*}}{{\\rotatebox[origin=c]{{90}}{{{model_name}}}}}  "
                        model_section += f"& {variant_name} & {variant['lat_mean']} & {variant['lat_worst']} & & {variant['inf_s']} & {variant['size_mb']} & {variant['layers']} & {variant['params']} & & {variant['prec_1']} & {variant['prec_5']} \\\\"
                   
                else:
                    model_section += f"    & {variant_name} & {variant['lat_mean']} & {variant['lat_worst']} & & {variant['inf_s']} & {variant['size_mb']} & {variant['layers']} & {variant['params']} & & {variant['prec_1']} & {variant['prec_5']} \\\\"
                
                model_section += "\n"
            latex_content += model_section

    full_latex = latex_header + latex_content + latex_footer
    return full_latex


In [53]:
def parse_markdown_data(file_content):
    models_data = file_content.split('# ')[1:]  # Divide por modelo y batch size, ignorando el primer split que estaría vacío
    data_summary = {}
    inf_s_data = {}

    # Procesar y almacenar inf/s para batch size 256 por separado
    best_inf_s = {}
    best_bs = {}
    for model_section in models_data:
        lines = model_section.split('\n')
        model_name_batch_size = lines[0].strip()
        model_name, _ = model_name_batch_size.split(' bs ')
        for line in lines:
            if '|' in line and 'Model' not in line and not all(char == '-' for char in line.replace('|', '').strip()):
                cols = [col.strip() for col in line.split('|')]
                if len(cols) > 1:
                    variant = cols[1]
                    if model_name not in best_inf_s:
                            best_inf_s[model_name] = {}
                            best_bs[model_name] = {}
                    best_inf_s[model_name][variant] = 0.0
                    best_bs[model_name][variant] = 0

    print(best_inf_s)

    for model_section in models_data:
        lines = model_section.split('\n')
        model_name_batch_size = lines[0].strip()
        model_name, batch_size_info = model_name_batch_size.split(' bs ')
        batch_size = int(batch_size_info.strip())

        #if batch_size == 256:
        for line in lines:
            if '|' in line and 'Model' not in line and not all(char == '-' for char in line.replace('|', '').strip()):
                cols = [col.strip() for col in line.split('|')]
                if len(cols) > 1:
                    variant = cols[1]
                    inf_s_str = (cols[2].split(' ')[0].replace('.','')).replace(',','.')
                    inf_s = float(inf_s_str)
                    if inf_s > best_inf_s[model_name][variant]:
                        best_inf_s[model_name][variant]  = inf_s
                        best_bs[model_name][variant] = batch_size
                        if model_name not in inf_s_data:
                            inf_s_data[model_name] = {}
                        if batch_size == 256:
                            inf_s_data[model_name][variant] = '$'+str(inf_s)+'$'
                        else: 
                            inf_s_data[model_name][variant] = '$'+str(inf_s)+'_{bs= '+str(batch_size)+'}$'

    # Procesar datos principales para batch size 1 y añadir inf/s de batch size 256 cuando corresponda
    for model_section in models_data:
        lines = model_section.split('\n')
        model_name_batch_size = lines[0].strip()
        model_name, batch_size_info = model_name_batch_size.split(' bs ')
        batch_size = int(batch_size_info.strip())

        if batch_size == 1:
            table_data = [line for line in lines if '|' in line and 'Model' not in line]

            data_summary[model_name] = []

            for data in table_data:
                if not all(char == '-' for char in data.replace('|', '').strip()):
                    cols = [col.strip() for col in data.split('|')]
                    if len(cols) > 1:
                        model_variant = cols[1]
                        inf_s = inf_s_data.get(model_name, {}).get(model_variant, 'N/A')  # Get inf/s from bs 256 data if available
                        lat_mean = cols[3].split(' / ')[0]
                        lat_worst = (cols[3].split(' / ')[1]).split(' ')[0]
                        lat_mean = lat_mean.strip()
                        lat_worst = lat_worst.strip()
                        size_mb = cols[4]
                        prec_1 = cols[5]
                        prec_5 = cols[6]
                        layers = cols[7]
                        params = cols[8]

                        model_data = {
                            'variant': model_variant,
                            'inf_s': inf_s,  # Added from bs 256 data
                            'lat_mean': lat_mean,
                            'lat_worst': lat_worst,
                            'size_mb': size_mb.strip(),
                            'prec_1': prec_1.strip(),
                            'prec_5': prec_5.strip(),
                            'layers': layers.strip(),
                            'params': params.strip()
                        }
                        data_summary[model_name].append(model_data)

    return data_summary


In [54]:
file_path = '../outputs/table_outputs/jetson_orin_nano/jetson_orin_nano_dynamic_batch.md'

# Abre y lee el archivo
with open(file_path, 'r', encoding='utf-8') as file:
    md_content = file.read()
# Parse the markdown content
parsed_data = parse_markdown_data(md_content)

print(parsed_data)

#for model_data in parsed_data.items():
#    for variants in model_data[1]:
#        print(model_data[0], variants)

{'mobilenet': {'Vanilla': 0.0, 'TRT_fp32': 0.0, 'TRT_fp16': 0.0, 'TRT_int8': 0.0}, 'resnet18': {'Vanilla': 0.0, 'TRT_fp32': 0.0, 'TRT_fp16': 0.0, 'TRT_int8': 0.0}, 'resnet34': {'Vanilla': 0.0, 'TRT_fp32': 0.0, 'TRT_fp16': 0.0, 'TRT_int8': 0.0}, 'resnet50': {'Vanilla': 0.0, 'TRT_fp32': 0.0, 'TRT_fp16': 0.0, 'TRT_int8': 0.0}, 'resnet101': {'Vanilla': 0.0, 'TRT_fp32': 0.0, 'TRT_fp16': 0.0, 'TRT_int8': 0.0}, 'resnet152': {'Vanilla': 0.0, 'TRT_fp32': 0.0, 'TRT_fp16': 0.0, 'TRT_int8': 0.0}}
{'mobilenet': [{'variant': 'Vanilla', 'inf_s': '230.8', 'lat_mean': '17.7', 'lat_worst': '31.5', 'size_mb': '13.6', 'prec_1': '72.02', 'prec_5': '90.62', 'layers': '104', 'params': '3487816'}, {'variant': 'TRT_fp32', 'inf_s': '688.3', 'lat_mean': '5.7', 'lat_worst': '11.9', 'size_mb': '14.4', 'prec_1': '72.02', 'prec_5': '90.62', 'layers': '57', 'params': '3469760'}, {'variant': 'TRT_fp16', 'inf_s': '1098.2', 'lat_mean': '3.6', 'lat_worst': '11.1', 'size_mb': '9.6', 'prec_1': '71.99', 'prec_5': '90.64', '

In [55]:
latex_table = generate_latex_table(parsed_data)
print(latex_table)


\begin{table}[]
    \centering
    \footnotesize
    \caption{Jetson Orin AGX}
    \begin{tabular}{@{}clccccccccccc@{}}
    \toprule
    & & \multicolumn{2}{c}{Latency {[}ms{]}} &  & Thr. {[}inf/s{]} &  \multicolumn{3}{c}{Model properties} &  & \multicolumn{2}{c}{Accuracy [\%]} \\ \cmidrule(lr){3-4}  \cmidrule(lr){7-9} \cmidrule(l){11-12} 
    & & ave.              & max.             &  &                       &  Size {[}MB{]}   & \# Layers   & \# Weights   &  & Top 1       & Top 5      \\ \midrule  \multirow{4}{*}{\rotatebox[origin=c]{90}{mobilenet}}  & Vanilla & 17.7 & 31.5 & & 230.8 & 13.6 & 104 & 3487816 & & 72.02 & 90.62 \\
    & TRT fp32 & 5.7 & 11.9 & & 688.3 & 14.4 & 57 & 3469760 & & 72.02 & 90.62 \\
    & TRT fp16 & 3.6 & 11.1 & & 1098.2 & 9.6 & 58 & 3469760 & & 71.99 & 90.64 \\
    & TRT int8 & 2.9 & 12.0 & & 1600.4 & 5.5 & 57 & 3469760 & & 71.45 & 90.29 \\
    \midrule 
    \multirow{4}{*}{\rotatebox[origin=c]{90}{resnet18}}  & Vanilla & 11.3 & 18.1 & & 343.8 & 44.7 & 53 & 